In [1]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Embedding Gecko (Multilingual)
* Refer to the link for more information about the embeddings.
* https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings
* https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession,
)

from vertexai.preview.generative_models import grounding
from vertexai.preview.language_models import TextEmbeddingModel


### Initalize Vertex AI

In [5]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper function

In [6]:
import numpy as np
import torch
import torch.nn as nn
from typing import List, Optional
from vertexai.vision_models import Image, MultiModalEmbeddingModel
#------------------------------------------------------------
def get_gecko_embeddings(image_path, contextual_text):

  model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")
  image = Image.load_from_file(image_path)

  embeddings = model.get_embeddings(
      image=image,
      contextual_text=contextual_text,
  )
  print(f"Image Embedding: {embeddings.image_embedding}")
  print(f"Text Embedding: {embeddings.text_embedding}")

  return embeddings

#------------------------------------------------------------
def dot_product(a, b):
  """ torch dot product similarity """

  return torch.dot(a, b)

#------------------------------------------------------------
def cosine_similarity(a, b):
  """ torch cosine similarity """

  cos = nn.CosineSimilarity(dim=0, eps=1e-6)
  output = cos(a, b)
  return output

#------------------------------------------------------------
def matrix_multiplication(a, b):
  """ torch matrix multiplication """

  if len(a.shape) == 1: a = a.unsqueeze(0)
  if len(b.shape) == 1: b = b.unsqueeze(0)

  a_norm = a / a.norm(dim=1)[:, None]
  b_norm = b / b.norm(dim=1)[:, None]
  return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

#------------------------------------------------------------
def print_similarity(input_embed, target_embed):
  import torch

  score0 = dot_product(torch.Tensor(input_embed),torch.Tensor(target_embed) )


  print(f"score0[{score0}]")



### Image Upload

In [13]:
from google.colab import files
from IPython.display import Image
uploaded = files.upload()

### Embedding

In [7]:
input = get_gecko_embeddings("./cha.jpg", "차승원" )

cha_1 = get_gecko_embeddings("./cha1.jpg", "차승원" )
cha_2 = get_gecko_embeddings("./cha2.png", "차승원" )
cha_3 = get_gecko_embeddings("./cha3.jpeg", "차승원" )
cha_4 = get_gecko_embeddings("./cha4.jpg", "차승원" )
cha_5 = get_gecko_embeddings("./cha5.jpg", "차승원" )
cha_6 = get_gecko_embeddings("./cha6.jpg", "차승원" )
cha_7 = get_gecko_embeddings("./cha7.jpg", "차승원" )

Image Embedding: [-0.0110581275, 0.0260190498, -0.0109927747, -0.0104005551, 0.00143533514, -0.0106042158, 0.00768690463, -0.00869297888, -0.026420176, 0.00124635838, -0.032614775, 0.0205467213, -0.0501487739, 0.0190449152, 0.00590474438, -0.0321571864, 0.0566953421, -0.0116167748, -0.004606192, -0.0176921077, -0.048973415, -0.00130457606, 0.0162395537, -0.0411570594, 0.0322006904, -0.0052534095, 0.0134332404, -0.003146759, 0.02454523, -0.00548020471, 0.00842039939, -0.016481109, -0.012831727, -0.0222176481, -0.0112459315, 0.00541423913, 0.00370100257, -0.0368632376, 0.0246528555, -0.0142671531, 0.0390058681, 0.00120328448, -0.00382684357, -0.0627599359, 0.0122669917, -0.0188628063, -0.0420733877, -1.11971585e-05, -0.00834318344, 0.0155079607, 0.0109182047, 0.0020247018, 0.00574933412, 0.00337652, -0.0283352, -0.0150974393, 0.0112143438, -0.0064094048, -0.0245238747, -0.0241360757, 0.0263237674, -0.0064714523, -0.0209921729, -0.0069064009, 0.00418386143, 0.00822832808, -0.000169705527,

In [8]:

print_similarity(cha_1.image_embedding, cha_1.image_embedding )
print_similarity(cha_1.image_embedding, cha_2.image_embedding )
print_similarity(cha_1.image_embedding, cha_3.image_embedding )
print_similarity(cha_1.image_embedding, cha_4.image_embedding )
print_similarity(cha_1.image_embedding, cha_5.image_embedding )
print_similarity(cha_1.image_embedding, cha_6.image_embedding )
print_similarity(cha_1.image_embedding, cha_7.image_embedding )


score0[0.9999999403953552]
score0[0.49574801325798035]
score0[0.5164413452148438]
score0[0.5419042110443115]
score0[0.5763565301895142]
score0[0.4180574119091034]
score0[0.45302069187164307]


In [9]:
others_1 = get_gecko_embeddings("./jung.jpeg", "정우성" )
others_2 = get_gecko_embeddings("./lee.png", "이경규" )
others_3 = get_gecko_embeddings("./um.jpg", "엄태구" )

Image Embedding: [-0.0457251742, 0.0522461794, -0.0214156453, -0.0108855972, 0.013806032, -0.00269525149, -0.0140629187, 0.001809904, -0.023792373, 0.0447806269, 0.00416699098, 0.0166029166, -0.00272524194, 0.0304000899, -0.00581048, 0.00372364512, 0.0492491536, -0.00264819758, -0.0398192368, 0.00190732663, -0.036898341, -0.00739618158, 0.0395344608, -0.0248018857, -0.00899772532, -0.00148362573, 0.00345017365, 0.00218759058, 0.00701445574, -0.0177048054, 0.0150419539, -0.00810969621, -0.00329857226, -0.0117672393, -0.00886154361, 0.00854304433, 0.0246781167, 0.00959833525, 0.0324314907, -0.0124780023, 0.0183427, -0.0120039908, 0.0187490489, -0.0474138297, 0.0184318386, -0.0121006966, -0.0121033704, -0.0235682484, -0.010101947, 0.0355405472, 0.0358973965, -0.0287048165, -0.0208789036, -0.0100750774, -0.0365860499, 0.00194371049, -0.0177602265, -0.0112741906, 0.00807169452, -0.0264925789, -0.00649836613, -0.0112399459, 0.0104287406, -0.000938623678, 0.00238176296, 0.0142276166, 0.003478

In [10]:
print_similarity(cha_1.image_embedding, others_1.image_embedding )
print_similarity(cha_1.image_embedding, others_2.image_embedding )
print_similarity(cha_1.image_embedding, others_3.image_embedding )

score0[0.49565717577934265]
score0[0.42423972487449646]
score0[0.5188356637954712]
